In [1]:
# Number of parameters for flan-t5 family: small 80M, base 250M, large 780M, xl 3B, xxl 11B
model_id = "google/flan-t5-large"

# https://huggingface.co/datasets/billsum
dataset_id = "billsum"

#### Setup

In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [3]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()

[03/09/25 18:56:38] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=611294;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=811727;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\botocore\credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\aayus\AppData\Local\sagemaker\sagemaker\config.yaml
2.241.0


[03/09/25 18:56:39] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=817549;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=828132;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\botocore\credentials.py#1278\1278]8;;\

In [4]:
import transformers
import datasets

print(transformers.__version__)
print(datasets.__version__)

[03/09/25 18:56:48] INFO     PyTorch version 2.5.1 available.                                          ]8;id=76598;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\datasets\config.py\config.py]8;;\:]8;id=139138;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\datasets\config.py#54\54]8;;\

4.48.3
3.3.0


### Preprocessing

#### Load Dataset

In [5]:
from datasets import load_dataset, load_from_disk

dataset = load_dataset(dataset_id)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 18949
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 3269
    })
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1237
    })
})

#### Preprocess data

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

prefix = "summarize: "
input_max_length = 1024
output_max_length = 128


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=input_max_length, truncation=True)
    labels = tokenizer(
        text_target=examples["title"], max_length=output_max_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 18949
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 3269
    })
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1237
    })
})


In [8]:
tokenized_dataset = dataset.map(
    preprocess_function, batched=True, remove_columns=["title", "text", "summary"]
)

Map:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [9]:
#tokenized_dataset.save_to_disk(f"billsum-t5-tokenized") 

#### Upload processed dataset to S3


In [10]:
pip install pyarrow


In [11]:
import pyarrow.parquet as pq
import pyarrow as pa
import fsspec
import sagemaker
# Initialize S3 session
sess = sagemaker.Session()
bucket = sess.default_bucket()

# Define S3 paths for training and validation datasets
s3_prefix = "huggingface/billsum-t5-summarization"
dataset_input_path = f"s3://{bucket}/{s3_prefix}"
train_input_path = f"{dataset_input_path}/train.parquet"
valid_input_path = f"{dataset_input_path}/test.parquet"  # `test` is used for validation

print("Train Input Path:", train_input_path)
print("Valid Input Path:", valid_input_path)

# Make sure they exist in S3
s3 = fsspec.filesystem("s3")
assert s3.exists(train_input_path), f"Train dataset not found at {train_input_path}"
assert s3.exists(valid_input_path), f"Validation dataset not found at {valid_input_path}"

# # Initialize S3
# s3 = fsspec.filesystem("s3")

# # Convert and upload train dataset
# train_table = pa.Table.from_pandas(tokenized_dataset["train"].to_pandas())
# train_s3_path = f"s3://{bucket}/huggingface/billsum-t5-summarization/train.parquet"

# with s3.open(train_s3_path, "wb") as f:
#     pq.write_table(train_table, f)
# print(f"Uploaded train dataset to {train_s3_path} 🚀")

# # Convert and upload test dataset
# test_table = pa.Table.from_pandas(tokenized_dataset["test"].to_pandas())
# test_s3_path = f"s3://{bucket}/huggingface/billsum-t5-summarization/test.parquet"

# with s3.open(test_s3_path, "wb") as f:
#     pq.write_table(test_table, f)
# print(f"Uploaded test dataset to {test_s3_path} 🚀")


[03/09/25 18:57:22] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=781719;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=371481;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\botocore\credentials.py#1278\1278]8;;\

Train Input Path: s3://sagemaker-us-east-1-767398054325/huggingface/billsum-t5-summarization/train.parquet
Valid Input Path: s3://sagemaker-us-east-1-767398054325/huggingface/billsum-t5-summarization/test.parquet


[03/09/25 18:57:27] INFO     Found credentials in shared credentials file: ~/.aws/credentials    ]8;id=501251;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\aiobotocore\credentials.py\credentials.py]8;;\:]8;id=849599;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\aiobotocore\credentials.py#621\621]8;;\

In [12]:
!aws s3 ls s3://sagemaker-us-east-1-767398054325/huggingface/billsum-t5-summarization/


2025-03-09 18:42:58    5603021 test.parquet
2025-03-09 18:42:49   33747084 train.parquet


###  Fine-tune on SageMaker with a Hugging Face Deep Learning Container

In [13]:
!pygmentize train.py 

import argparse
import logging
import os

import evaluate
import numpy as np
from datasets import load_from_disk
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

rouge = evaluate.load("rouge")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}


if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    # hyperparameters are passed 

In [14]:
hyperparameters = {
    "epochs": 1,
    "learning-rate": 1e-6,
    "train-batch-size": 1,
    "eval-batch-size": 8,
    "model-name": model_id,
}

In [ ]:
from sagemaker.huggingface import HuggingFace
import os
from dotenv import load_dotenv

load_dotenv()
role = os.getenv("SAGEMAKER_ROLE")
bucket = os.getenv("S3_BUCKET_NAME")

huggingface_estimator = HuggingFace(
    role=role,  # Use the IAM role
    entry_point="train.py",
    dependencies=["requirements.txt"],
    hyperparameters=hyperparameters,
    transformers_version="4.26.0",
    pytorch_version="1.13.1",
    py_version="py39",
    instance_type="ml.p3.2xlarge",
    instance_count=1,
    distribution={"smdistributed": {"dataparallel": {"enabled": True}}}
)


[03/09/25 18:57:35] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=688079;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=637780;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\botocore\credentials.py#1278\1278]8;;\

[03/09/25 18:57:36] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=217833;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=187688;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\botocore\credentials.py#1278\1278]8;;\

In [16]:
huggingface_estimator.fit({"train": train_input_path, "valid": valid_input_path})


                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=708241;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\sagemaker\telemetry\telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=939139;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\sagemaker\telemetry\telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=514506;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=937349;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\sagemaker\image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

[03/09/25 18:57:41] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=206193;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=234777;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\sagemaker\image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

                    INFO     Creating training-job with name:                                       ]8;id=489473;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=664396;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\sagemaker\session.py#1042\1042]8;;\
                             huggingface-pytorch-training-2025-03-09-13-27-36-718                                  

[03/09/25 18:57:43] ERROR    Please check the troubleshooting guide for common errors:              ]8;id=195979;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=867203;file://C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\sagemaker\session.py#1050\1050]8;;\
                             https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-t                
                             roubleshooting.html#sagemaker-python-sdk-troubleshooting-create-traini                
                             ng-job                                                                                

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 huggingface_estimator.fit({"train": train_input_path, "valid": valid_input_path})            │
│   2                                                                                              │
│                                                                                                  │
│ C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\sagemaker\telemetry\tel │
│ emetry_logging.py:167 in wrapper                                                                 │
│                                                                                                  │
│   164 │   │   │   │   │   caught_ex = e                                                          │
│   165 │   │   │   │   finally:                                                                   │
│   166 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 167 │   │   │   │   │   │   raise caught_ex                                                    │
│   168 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   169 │   │   │   else:                                                                          │
│   170 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\sagemaker\telemetry\tel │
│ emetry_logging.py:138 in wrapper                                                                 │
│                                                                                                  │
│   135 │   │   │   │   start_timer = perf_counter()                                               │
│   136 │   │   │   │   try:                                                                       │
│   137 │   │   │   │   │   # Call the original function                                           │
│ ❱ 138 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   139 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   140 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   141 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\sagemaker\workflow\pipe │
│ line_context.py:346 in wrapper                                                                   │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\

In [ ]:
huggingface_estimator.model_data 

### Deploy on sagemaker with a hugging face deep leraning container

In [17]:
huggingface_predictor = huggingface_estimator.deploy(
    initial_instance_count=1, instance_type="ml.p3.2xlarge"
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 huggingface_predictor = huggingface_estimator.deploy(                                        │
│   2 │   initial_instance_count=1, instance_type="ml.p3.2xlarge"                                  │
│   3 )                                                                                            │
│   4                                                                                              │
│                                                                                                  │
│ C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\sagemaker\estimator.py: │
│ 1689 in deploy                                                                                   │
│                                                                                                  │
│   1686 │   │   """                                                                               │
│   1687 │   │   removed_kwargs("update_endpoint", kwargs)                                         │
│   1688 │   │   is_serverless = serverless_inference_config is not None                           │
│ ❱ 1689 │   │   self._ensure_latest_training_job()                                                │
│   1690 │   │   self._ensure_base_job_name()                                                      │
│   1691 │   │                                                                                     │
│   1692 │   │   jumpstart_base_name = get_jumpstart_base_name_if_jumpstart_model(                 │
│                                                                                                  │
│ C:\Users\aayus\OneDrive\Desktop\ML\LegalEase\legalease\Lib\site-packages\sagemaker\estimator.py: │
│ 2226 in _ensure_latest_training_job                                                              │
│                                                                                                  │
│   2223 │   ):                                                                                    │
│   2224 │   │   """Placeholder docstring"""                                                       │
│   2225 │   │   if self.latest_training_job is None:                                              │
│ ❱ 2226 │   │   │   raise ValueError(error_message)                                               │
│   2227 │                                                                                         │
│   2228 │   delete_endpoint = removed_function("delete_endpoint")                                 │
│   2229                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Estimator is not associated with a training job

In [18]:
test_data = {"inputs": f"{prefix}: {dataset['test'][10]}"}

In [19]:
prediction = huggingface_predictor.predict(test_data)
print(prediction)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 prediction = huggingface_predictor.predict(test_data)                                        │
│   2 print(prediction)                                                                            │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'huggingface_predictor' is not defined

In [20]:
 huggingface_predictor.delete_endpoint() 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 huggingface_predictor.delete_endpoint()                                                      │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'huggingface_predictor' is not defined